In [1]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.Pharm2D import Gobbi_Pharm2D
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LinearRegression, Ridge, LassoCV, Lasso, ElasticNetCV
from sklearn.svm import LinearSVR
from src.data import make_dataset
from src.features import build_features,build_targets
from src.models import predict_model, train_model, split_data, bitranking
import numpy as np
import matplotlib.pyplot as plt

RDKit WARNING: [21:05:23] Enabling RDKit 2019.09.2 jupyter extensions


In [2]:
# OPTIONAL: Load the "autoreload" extension so that code can change
%load_ext autoreload
# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2
seed=42

In [3]:
# Load data
molecules=make_dataset.load()

In [4]:
# Build the features
#X=build_features.build(molecules,fpSize=512)
X=build_features.buildPharm(molecules)

IndexError: distance bin not found: feats: ['Acceptor', 'Aromatic', 'Donor']; dists=[1, 5, 1]; bins=[(0, 2), (2, 5), (5, 8)]; scaffolds: [0, [(0,), (1,), (2,)], 0, [(0, 0, 0), (0, 0, 1), (0, 1, 0), (0, 1, 1), (0, 1, 2), (0, 2, 1), (0, 2, 2), (1, 0, 0), (1, 0, 1), (1, 0, 2), (1, 1, 0), (1, 1, 1), (1, 1, 2), (1, 2, 0), (1, 2, 1), (1, 2, 2), (2, 0, 1), (2, 0, 2), (2, 1, 0), (2, 1, 1), (2, 1, 2), (2, 2, 0), (2, 2, 1), (2, 2, 2)], 0]

In [ ]:
# Plot one of the feature vectors
m1=[i for i in range(len(molecules)) if molecules[i].GetProp("NAME")=='91E2']
m1=m1[0]
m2=[i for i in range(len(molecules)) if molecules[i].GetProp("NAME")=='91F2']
m2=m2[0]
plt.plot(X[m1]-X[m2])
plt.ylabel("%s - %s"%(molecules[m1].GetProp("NAME"),molecules[m2].GetProp("NAME")))
plt.show()
print("Different at %d/%d"%(sum(X[m1]!=X[m2]),len(X[m1])))

In [7]:
# Build the targets
keep=['H960-003','Amb-751']
keep=None
(y,aptamers)=build_targets.build_fold(molecules,keep)

Column names are Target, H960-266, H960-319, H960-281, H960-850, H960-892, H960-735, H960-425, H960-940, H960-613, H960-251, H960-003, H960-875, H960-650, H960-506, H960-172, H960-050, H960-594, H960-228, H960-488, H960-629, H960-668, H960-374, H960-561, H960-156, H960-922, H960-962, H960-843, H960-616, H960-617, H960-505, H960-072, H960-724, H960-315, H960-354, H960-939, H960-256, H960-920, H960-356, H960-198, H960-337, H960-540, NSRef-413, NSRef-630, NS-104, Amb-767, Amb-563, Amb-113, Amb-751, Amb-816, Amb-720, Amb-318, Amb-6319
Processed 101 targets.
['1A4', '1C5', '1C7', '1F7', '101B7', '101C7', '101D11', '101D7', '101D9', '101E10', '101E6', '101F11', '101F2', '101F7', '101G6', '101H6', '111E3', '111H2', '111H7', '11D3', '21G8', '31B11', '31B9', '31C10', '31C2', '31C3', '31C8', '31D4', '31D7', '31D8', '31E10', '31E3', '31E4', '31E7', '31E9', '31F10', '31H10', '41A11', '41C4', '41C5', '41C7', '41D10', '41D4', '41D7', '41E10', '41E2', '41E3', '41E4', '41E7', '41E9', '41F10', '41F2', 

In [ ]:
# Plot the target distribution
plt.hist(y[0])
plt.yscale('log')
plt.show()

In [ ]:
# Create train/test sets
X_train, X_test, y_train, y_test, ind_train, ind_test = split_data.split(0.4,X,np.transpose(y),seed)
y_test=np.transpose(y_test)
y_train=np.transpose(y_train)
mol_test=[molecules[x] for x in ind_test]
mol_train=[molecules[x] for x in ind_train]

In [ ]:
# Test models to (manually) choose best one based on cross-validation over training set
test_aptamer=1
max_features=30
train_model.cv_model(LinearRegression(), X_train, y_train[test_aptamer])
train_model.cv_model_sfm(LinearRegression(), X_train, y_train[test_aptamer],max_features=max_features)
train_model.cv_model(LinearSVR(C=.001,max_iter=10000,epsilon=0), X_train, y_train[test_aptamer])
train_model.cv_model(LassoCV(), X_train, y_train[test_aptamer])
train_model.cv_model(ElasticNetCV(), X_train, y_train[test_aptamer])

In [ ]:
# Train model using best model determined above
model=train_model.train_multi(LinearSVR(C=.000001,max_iter=10000,epsilon=0.1,random_state=seed),X_train,y_train)

In [ ]:
# Test model
print("------")
print(model[0])
print("Train:")
yp_train=predict_model.predict_reg(model,X_train,y_train,mol_train)
yp_train=np.transpose(yp_train)
print("Test:")
yp_test=predict_model.predict_reg(model,X_test,y_test,mol_test)
yp_test=np.transpose(yp_test)

for i in range(len(y_train)):
    plt.scatter(y_train[i] ,yp_train[i],label="Train",s=1)
ax=plt.gca()
#ax.set_yscale('log')
#ax.set_xscale('log')
plt.xlabel("Measured - log(fold)")
plt.ylabel("Predicted")
plt.title("Train")
plt.show()

for i in range(len(y_test)):
    plt.scatter(y_test[i],yp_test[i] ,label="Test",s=1)
ax=plt.gca()
#ax.set_yscale('log')
#ax.set_xscale('log')
plt.xlabel("Measured - log(fold)")
plt.ylabel("Predicted")
plt.title("Test")
plt.show()

In [ ]:
# Pharmacore fingerprints
mol = Chem.MolFromSmiles('OCC(=O)CCCN')
mol=molecules[0]
print(Chem.MolToMolBlock(mol)) 
Chem.SanitizeMol(mol)
mol = Chem.MolFromSmiles(AllChem.MolToSmiles(mol))
print(Chem.MolToMolBlock(mol)) 
AllChem.Compute2DCoords(mol)
print(AllChem.MolToSmiles(mol))
fp=build_features.buildGobbi([mol])
print(len(fp[0]),fp[0].GetNumOnBits())
for bit in list(fp[0].GetOnBits()):
    print(Gobbi_Pharm2D.factory.GetBitDescription(bit))

In [10]:
# Fragment fingerprints
fp, fcat=build_features.buildFragment(molecules)
print(len(fp),len(fp[0]),fp[0].GetNumOnBits())
X=fp

Num functional groups: 39
Have 48716 fragments in library
960 48716 154


In [12]:
# Rank fragments based on observed activity
ntop=10
for i in range(len(aptamers)):
    print(aptamers[i])
    bitranking.getFragRanks(fcat, fp, [ 1 if yi>0.3 else 0 for yi in y[i]],ntop=ntop)



H960-266
Num of active targets: 7
585 0.023 43 6 cnc(c)nc<=O>
520 0.022 48 6 c<=O>nc(c)n
488 0.021 53 6 ccnc<=O>
19758 0.021 23 5 cc(n)nccC
610 0.021 56 6 c<=O>ccncn
613 0.021 57 6 cnccc<=O>[nH]
19687 0.020 25 5 Cccncn
540 0.020 61 6 cncnc<=O>
529 0.020 62 6 c<=O>ccnc
526 0.020 64 6 nccc<=O>[nH]
H960-319
Num of active targets: 28
19775 0.045 16 13 c<=O>ccncnc
737 0.040 22 13 c<=O>cc(n)ncn
719 0.040 22 13 c<=O>nc(c)ncn
819 0.040 22 13 cncncnC
696 0.040 22 13 c<=O>cc(nC)nc
19685 0.040 16 12 cncn(c<=O>)c
19657 0.040 16 12 ccn(c)c<=O>
19693 0.040 16 12 cc<=O>n(c)cc
19692 0.040 16 12 cccn(c)c<=O>
19683 0.040 16 12 ccn(c<=O>)cn
H960-281
Num of active targets: 22
585 0.039 36 13 cnc(c)nc<=O>
520 0.037 41 13 c<=O>nc(c)n
488 0.035 46 13 ccnc<=O>
610 0.034 49 13 c<=O>ccncn
613 0.034 50 13 cnccc<=O>[nH]
540 0.032 54 13 cncnc<=O>
529 0.032 55 13 c<=O>ccnc
523 0.032 72 14 cnc(c)n
526 0.031 57 13 nccc<=O>[nH]
490 0.030 62 13 c<=O>ccn
H960-850
Num of active targets: 11
253 0.035 108 11 cccccnc
19657 

In [ ]:
print(X.shape())

In [ ]:
X=np.array(fp)


In [ ]:
print(X.shape)

